<a href="https://colab.research.google.com/github/bpawnzZ/colabNotebooks/blob/main/StableDiffusionWebUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [stable-diffusion-webui-colab](https://github.com/dsymbol/stable-diffusion-webui-colab) Notebook

**Remember to use the free resource fairly and not to run a free session for over 12 hours.** You can get [Colab Pro/Pro+](https://colab.research.google.com/signup/pricing) for longer runtimes.

Stable Diffusion related issues post an issue [here](https://github.com/AUTOMATIC1111/stable-diffusion-webui/issues/new)  
Google Colab related issues post an issue [here](https://github.com/dsymbol/stable-diffusion-webui-colab/issues/new)

**Credits**:  
[AUTOMATIC1111](https://github.com/AUTOMATIC1111)  
[K4YT3X](https://github.com/K4YT3X)

In [ ]:
#@markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/Google-Drive-Logo.png" height="50" alt="Google-Drive-Logo"/></center>
#@markdown <center><h3>Connect to Google Account</h3></center><br>

from pathlib import Path
from contextlib import suppress
from sys import exit as end
from google.colab import output
from google.colab.drive import mount, flush_and_unmount as unmount

MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
FORCE_REMOUNT = True #@param {type:"boolean"}
CUSTOM_MOUNT_POINT = False #@param {type:"boolean"}
DEBUG_MODE = False #@param {type:"boolean"}

if CUSTOM_MOUNT_POINT:
  Custom_Mount_Path = input('Please enter Custom Mount Path:\n')
  if not Custom_Mount_Path:
    end("Path Error: Path can't be blank!")
  elif not Path(Custom_Mount_Path).exists():
    end("Path Error: Provided Custom path not found! Check if you mounted/connected it or if you having reading/writing permission?")
  elif not Path(Custom_Mount_Path).is_dir():
    end("Path Error: Given Path is not a valid directory.")
else:
  Custom_Mount_Path = '/content/drive'

mount._DEBUG = DEBUG_MODE

if MODE == "MOUNT":
  print("Please Wait! Mounting...")
  mount(Custom_Mount_Path, force_remount= FORCE_REMOUNT)
  if not DEBUG_MODE:
    output.clear()
  print(f'Mounted at {Custom_Mount_Path}')
elif MODE == 'UNMOUNT':
  with suppress(ValueError):
    print('Please Wait! UNMOUNT in progress...')
    unmount()
  get_ipython().system_raw('rm -rf /root/.config/Google/DriveFS')

Mounted at /content/drive


In [ ]:
#@markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/Setup_img.png" height="50" alt="Setup-Logo"/></center>
#@markdown <center><h3><b>Setup & Configure Environment</b></h3></center><br>

from sys import exit as end
from pathlib import Path, PurePath
from time import sleep
from requests import get as get_url
from urllib.parse import urlparse as validate_url
from google.colab import output
from json import dumps as dump_json

current_version = "v5.1.0"
do_normal_setup = "1"

# Check for latest version
try:
    json_data = get_url("https://assets.thecaduceus.eu.org/MCT/json/mct.json").json()
    print("Connected with CDN.")
    if current_version == json_data["version"]:
        print(f'Great Job! you are using latest version: {json_data["version"]}')
    else:
        print(f'Oh-No! you are not using latest version and missing out important fixes & features. Latest version is {json_data["version"]}, use latest notebook available here:\n{json_data["notebookLink"]}')
        sleep(10)
except Exception as error_code:
    end(f'Failed to connect with CDN!\nTraceback:\n{error_code}')

# Save json data locally
with open('mct.json', 'w+') as mct_json:
    mct_json.write(dump_json(json_data))

# Flavor Flags
Stable = ""
Beta = "-s beta"

# User Input
INSTALL_RCLONE_FLAVOR = Stable #@param ["Stable", "Beta"] {type:"raw"}
FORCE_SETUP = True #@param {type:"boolean"}
CUSTOM_SH_FILE = False #@param {type:"boolean"}

if CUSTOM_SH_FILE:

    Custom_sh_Source = input("Provide the Custom Install.sh file source, it can be URL or File Path:\n")
    use_sudo = input("Do you want to execute the .sh file as sudo? 1 = Yes and 0 = No:\n")

    if use_sudo not in ['0','1']:
        end("Unable to check for latest version!")
    elif not Custom_sh_Source:
        end("Custom sh file source can't be blank!")
    elif all([validate_url(Custom_sh_Source).scheme, validate_url(Custom_sh_Source).netloc]):
        try:
            get_url(Custom_sh_Source)
            if use_sudo == '1':
                !curl {Custom_sh_Source} | sudo bash
                print('Execution completed as sudo!')
            elif use_sudo == '0':
                !curl {Custom_sh_Source} | bash
                print('Execution completed!')
        except ConnectionError:
            end("Unable to connect with given sh file URL.")
        except Exception as error_code:
            end(f"Failed to complete the execution of .sh file!\nTraceback:\n{error_code}")
    elif not Path(Custom_sh_Source).exists():
        end("Provided Custom sh file path not exist!")
    elif not Path(Custom_sh_Source).is_file():
        end("Provided Custom sh file path should of a particular file not directory/folder.")
    elif not PurePath(Custom_sh_Source).suffix == '.sh':
        end("File format should be .sh")
    else:
        try:
            if use_sudo == '1':
                !sudo bash "$Custom_sh_Source"
                print("Execution completed as sudo.")
            else:
                !bash "$Custom_sh_Source"
                print("Execution complete.")
        except Exception as error_code:
            end(f"Failed to complete the execution of .sh file!\nTraceback:\n{error_code}")


    # Ask for Normal Setup
    do_normal_setup = input('Do you want to run Normal Setup also? 1 = Yes and 0 = No:\n')

    if do_normal_setup not in ['0','1']:
        end("Input Error: Value should be 1 or 0")

# Force Setup
if FORCE_SETUP:
    if Path("/content/setup.sh").exists():
        !rm {"/content/setup.sh"}
    print("Starting Force Setup...")
    sleep(4)
    !curl "https://rclone.org/install.sh" | sudo bash {INSTALL_RCLONE_FLAVOR}

if do_normal_setup == '1' and not Path("/content/setup.sh").exists():
    try:
        get_url(json_data["setupFile"])
        !curl {json_data["setupFile"]} | sudo bash {INSTALL_RCLONE_FLAVOR}
        !curl {json_data["setupFile"]} > "/content/setup.sh"
        output.clear()
        print("Setup completed!")
    except ConnectionError:
        print('Unable to get setup file!')
    except Exception as error_code:
        print(f'Setup instructions seems to be having mistake(s)!\nTraceback:\n{error_code}')
elif do_normal_setup == '1' and Path("/content/setup.sh").exists():
    print("Setup file exist and configured already!")

Setup completed!


In [ ]:
#@markdown <br><center><img src="https://assets.thecaduceus.eu.org/MCT/img/rclone_logo.png" height="50" alt="Rclone-Logo"/></center>
#@markdown <center><h3><b>Fetch Previous MCT Config from My Drive</b></h3></center><br>

from sys import exit as end
from pathlib import Path, PurePath

# Where user mounted Google Drive?
try:
  CMP = PurePath(Custom_Mount_Path)
except NameError:
  CMP = '/content/drive'

DRIVE_CONFIG_DIR = f"{CMP}/MyDrive/MCT-Config/"

# Check for Setup & Google Account
if not Path("/content/setup.sh").exists():
  end('Please download setup first!')
elif not Path(CMP).exists():
  end('Not Found: Google Drive is not connected!')

#@markdown ---
CUSTOM_IMPORT_PATH = False #@param {type:"boolean"}
CUSTOM_EXPORT_PATH = False #@param {type:"boolean"}

# Import Configuration(s)

if CUSTOM_IMPORT_PATH:
  Custom_Path = input('Enter the directory path from where configuration file(s) needs to be imported:\n')
  # Check Path
  if not Custom_Path:
    end("Path Error: Path can't be blank!")
  elif not Path(Custom_Path).exists():
    end("Path Error: Provided Custom path not found! Check if you mounted/connected it or if you having reading/writing permission?")
  elif not Path(Custom_Path).is_dir():
    end("Path Error: Given Path is not a valid directory.")
  elif not Path(f'{CMP}/MyDrive/MCT-Config/').exists():
    !rclone mkdir "$DRIVE_CONFIG_DIR"
    print('Created: MCT-Config [folder]')

  Custom_Path = PurePath(Custom_Path) # Remove any extra '/'

  # Paths
  IMPORT_TO = DRIVE_CONFIG_DIR
  IMPORT_RCLONE_CONF = f"{Custom_Path}/rclone.conf"
  IMPORT_ACCOUNTS_ZIP = f"{Custom_Path}/accounts.zip"
  IMPORT_ACCOUNTS_FOLDER = f"{Custom_Path}/accounts/"
  ZIP_ACCOUNTS_FOLDER = f"{CMP}/MyDrive/MCT-Config/accounts" # accounts is name of zip, zip file will be saved in MCT-Config
  IMPORT_MEGA_JSON = f"{Custom_Path}/mega.json"
  IMPORT_GOFILE_JSON = f"{Custom_Path}/gofile.json"
  IMPORT_TELEGRAM_JSON = f"{Custom_Path}/telegram.json"
  IMPORT_SESSION_FILES = list(Path(Custom_Path).glob("*.session"))
  # Check 'rclone.conf' file

  if Path(IMPORT_RCLONE_CONF).exists():

    is_import = input('Found: rclone.conf [file] | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_RCLONE_CONF" "$IMPORT_TO"
      print('Successfully imported "rclone.conf" file! (1)')
    else:
      print('Not Found: rclone.conf [file]')

  # Check 'Service Accounts' file/folder

  if Path(IMPORT_ACCOUNTS_ZIP).exists() and Path(IMPORT_ACCOUNTS_FOLDER).exists():

    choice = input('Found both! "accounts.zip" [file] & "accounts" [folder]\n Select one: 0 = accounts.zip [file] or 1 = accounts [folder] or 2 = Ignore\n')

    if choice not in ['0','1','2']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif choice == '1':
      !zip "$ZIP_ACCOUNTS_FOLDER" "$IMPORT_ACCOUNTS_FOLDER"
      print('Successfully imported "accounts.zip" file! (2)')
    elif choice == '0':
      !rclone copy "$IMPORT_ACCOUNTS_ZIP" "$IMPORT_TO"
      print('Successfully imported "accounts.zip" file! (1)')
  elif Path(IMPORT_ACCOUNTS_ZIP).exists():

    is_import = input('Found: accounts.zip [file] | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_ACCOUNTS_ZIP" "$IMPORT_TO"
      print('Successfully imported "accounts.zip" file! (1)')
  elif Path(IMPORT_ACCOUNTS_FOLDER).exists():

    is_import = input('Found: accounts [folder] | Do you want to import ? 0=No & 1=Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !zip "$ZIP_ACCOUNTS_FOLDER" "$IMPORT_ACCOUNTS_FOLDER"
      print('Successfully imported "accounts.zip" file! (2)')
  else:
    print('Not Found: accounts.zip [file] or accounts [folder]')

  # Check 'mega.json' file

  if Path(IMPORT_MEGA_JSON).exists():

    is_import = input('Found: mega.json | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_MEGA_JSON" "$IMPORT_TO"
      print('Successfully imported "mega.json" file! (1)')

  else:
    print('Not Found: mega.json [file]')

  # Check 'gofile.json' file

  if Path(IMPORT_GOFILE_JSON).exists():

    is_import = input('Found: gofile.json | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_GOFILE_JSON" "$IMPORT_TO"
      print('Successfully imported "gofile.json" file! (1)')

  else:
    print('Not Found: gofile.json [file]')

  # Check 'telegram.json' file

  if Path(IMPORT_TELEGRAM_JSON).exists():

    is_import = input('Found: telegram.json | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      !rclone copy "$IMPORT_TELEGRAM_JSON" "$IMPORT_TO"
      print('Successfully imported "telegram.json" file! (1)')

  else:
    print('Not Found: telegram.json [file]')

  # Check '*.session' files

  if len(IMPORT_SESSION_FILES) != 0:

    is_import = input('Found: Telegram session files | Do you want to import? 0 = No & 1 = Yes:\n')

    if is_import not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_import == '1':
      for IMPORT_SESSION_FILE in IMPORT_SESSION_FILES:
        !rclone copy "$IMPORT_SESSION_FILE" "$IMPORT_TO"
      print("Successfully imported telegram session files! (1)")

  else:
    print('Not Found: .session [file]')

# Restore Configuration(s) from MCT-Config folder

if not Path(DRIVE_CONFIG_DIR).exists():
  end('Not Found: MCT-Config [folder]')
else:
  print('Found: MCT-Config [folder]')

# Paths
ROOT = "/content/"
DRIVE_RCLONE_CONF = f"{CMP}/MyDrive/MCT-Config/rclone.conf"
ROOT_RCLONE_CONF = "/root/.config/rclone/"
DRIVE_ACCOUNTS_ZIP = f"{CMP}/MyDrive/MCT-Config/accounts.zip"
DRIVE_MEGA_JSON = f"{CMP}/MyDrive/MCT-Config/mega.json"
DRIVE_GOFILE_JSON = f"{CMP}/MyDrive/MCT-Config/gofile.json"
DRIVE_TELEGRAM_JSON = f"{CMP}/MyDrive/MCT-Config/telegram.json"
DRIVE_SESSION_FILES = list(Path(f"{CMP}/MyDrive/MCT-Config/").glob("*.session"))

# Check 'rclone.conf' file

if Path(DRIVE_RCLONE_CONF).exists():
  !rclone copy "$DRIVE_RCLONE_CONF" "$ROOT_RCLONE_CONF"
  print('Restored: rclone.conf [file]')
else:
  print('Not Found: rclone.conf [file]')

# Check 'Service Accounts' file

if Path(DRIVE_ACCOUNTS_ZIP).exists():
  !rclone copy "$DRIVE_ACCOUNTS_ZIP" "$ROOT"
  print('Restored: accounts.zip [file]')
else:
  print('Not Found: accounts.zip [file]')

# Check 'mega.json' file

if Path(DRIVE_MEGA_JSON).exists():
  !rclone copy "$DRIVE_MEGA_JSON" "$ROOT"
  print('Restored: mega.json [file]')
else:
  print('Not Found: mega.json [file]')

# Check 'gofile.json' file

if Path(DRIVE_GOFILE_JSON).exists():
  !rclone copy "$DRIVE_GOFILE_JSON" "$ROOT"
  print('Restored: gofile.json [file]')
else:
  print('Not Found: gofile.json [file]')

# Check 'telegram.json' file

if Path(DRIVE_TELEGRAM_JSON).exists():
  !rclone copy "$DRIVE_TELEGRAM_JSON" "$ROOT"
  print('Restored: telegram.json [file]')
else:
  print('Not Found: telegram.json [file]')

# Check '*.session' files

if len(DRIVE_SESSION_FILES) != 0:
  for DRIVE_SESSION_FILE in DRIVE_SESSION_FILES:
    !rclone copy "$DRIVE_SESSION_FILE" "$ROOT"
  print('Restored: *.session [files]')
else:
  print('Not Found: *.session [files]')

if CUSTOM_EXPORT_PATH:

  #Paths
  EXPORT_TO = f"{Custom_Path}/MCT-Config/"

  Custom_Path = input('Enter the directory path to where configuration file(s) needs to be exported:\n')

  if not Custom_Path:
    end("Path Error: Path can't be blank!")
  elif not Path(Custom_Path).exists():
    end("Path Error: Path not found!")
  elif not Path(Custom_Path).is_dir():
    end("Path Error: Path is not a valid directory.")
  elif not Path(DRIVE_CONFIG_DIR):
    !rclone mkdir "$DRIVE_CONFIG_DIR"

  # Check 'rclone.conf' file

  if Path(DRIVE_RCLONE_CONF).exists():

    is_export = input('Found: rclone.conf [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_RCLONE_CONF" "$EXPORT_TO"
      print('Export: rclone.conf [file] (1)')
  else:
    print('Not Found: rclone.conf [file]')

    # Check 'Service Accounts' file

  if Path(DRIVE_ACCOUNTS_ZIP).exists():

    is_export = input('Found accounts.zip [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_ACCOUNTS_ZIP" "$EXPORT_TO"
      print('Export: accounts.zip [file] (1)')
  else:
    print('Not Found: accounts.zip [file]')

  # Check 'mega.json' file

  if Path(DRIVE_MEGA_JSON).exists():

    is_export = input('Found mega.json [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_MEGA_JSON" "$EXPORT_TO"
      print('Export: mega.json [file] (1)')
  else:
    print('Not Found: mega.json [file]')

  # Check 'gofile.json' file

  if Path(DRIVE_GOFILE_JSON).exists():

    is_export = input('Found gofile.json [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0', '1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_GOFILE_JSON" "$EXPORT_TO"
      print('Export: gofile.json [file] (1)')
  else:
    print('Not Found: gofile.json [file]')

  # Check 'telegram.json' file

  if Path(DRIVE_TELEGRAM_JSON).exists():

    is_export = input('Found telegram.json [file] | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      !rclone copy "$DRIVE_TELEGRAM_JSON" "$EXPORT_TO"
      print('Export: telegram.json [file] (1)')
  else:
    print('Not Found: telegram.json [file]')

  if len(DRIVE_SESSION_FILES) != 0:

    is_export = input('Found Telegram session files | Do you want to export? 0 = No & 1 = Yes:\n')

    if is_export not in ['0','1']:
      end('Input Error: Invalid input parameter! Please enter 0 or 1 as input.')
    elif is_export == '1':
      for DRIVE_SESSION_FILE in DRIVE_SESSION_FILES:
        !rclone copy "$DRIVE_SESSION_FILE" "$EXPORT_TO"
      print("Export: Telegram session files (1)")
  else:
    print("Not Found: Telegram session [files]")

Found: MCT-Config [folder]
2024/02/24 06:47:39 NOTICE: Config file "/root/.config/rclone/rclone.conf" not found - using defaults
Restored: rclone.conf [file]
Not Found: accounts.zip [file]
Not Found: mega.json [file]
Not Found: gofile.json [file]
Not Found: telegram.json [file]
Not Found: *.session [files]


In [ ]:
#@title ## Step 0: Runtime Selection

#@markdown This step is manual. **You'll need to select a GPU runtime** on the menu bar at the top, select:

#@markdown > Runtime > Change runtime type > Hardware accelerator

#@markdown Change "Hardware accelerator" to GPU and click "Save." After you're done, execute this cell to see the detailed information about the GPU allocated to you (optional).

!nvidia-smi

Sat Feb 24 06:47:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#@title ## Step 1: Install stable-diffusion-webui Dependencies

!apt-get update -y
!apt-get install python3.10 build-essential

!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 2

!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10

!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui/
%cd stable-diffusion-webui

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
python3.10 is already the newest version (3.10.12-1~22.04.3).
The following p

In [ ]:
#@title ## Step 2: Install and Run stable-diffusion-webui

#@markdown Once installation is done you will get a link to gradio.app, open it.

!python launch.py --share --gradio-debug

Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Version: v1.7.0
Commit hash: cf2772fab0af5573da775e7437e6acdca424f26e
Faceswaplab : Use GPU requirements
Checking faceswaplab requirements
0.008214740999392234
ReActor preheating... Device: CUDA
Launching Web UI with arguments: --share --gradio-debug
2024-02-24 12:31:58.700410: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-24 12:31:58.700456: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-24 12:31:58.701883: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-24 12:31:59.944651: W tensorflow/compiler/tf2ten

In [ ]:
!rclone sync /content/stable-diffusion-webui/outputs 1d:output3 --s3-upload-concurrency 32 --s3-chunk-size 128000 --fast-list --transfers 8 -P

In [ ]:
!rclone sync 1d:AI/automatic1111/models/ckpt /content/stable-diffusion-webui/models/Stable-diffusion --s3-upload-concurrency 32 --s3-chunk-size 128000 --fast-list -P

!rclone sync 1d:AI/automatic1111/models/lora /content/stable-diffusion-webui/models/Lora --s3-upload-concurrency 32 --s3-chunk-size 128000 --fast-list -P

!rclone sync 1d:AI/automatic1111/models/insightface /content/stable-diffusion-webui/models/insightface --s3-upload-concurrency 32 --s3-chunk-size 128000 --fast-list -P

!rclone sync 1d:AI/automatic1111/models/codeformer /content/stable-diffusion-webui/models/Codeformer --s3-upload-concurrency 32 --s3-chunk-size 128000 --fast-list -P

!rclone sync 1d:AI/automatic1111/models/esrgan /content/stable-diffusion-webui/models/ESRGAN --s3-upload-concurrency 32 --s3-chunk-size 128000 --fast-list -P

!rclone sync 1d:AI/automatic1111/models/gfpgan /content/stable-diffusion-webui/models/GFPGAN --s3-upload-concurrency 32 --s3-chunk-size 128000 --fast-list -P

!rclone sync 1d:AI/automatic1111/embeddings /content/stable-diffusion-webui/embeddings --s3-upload-concurrency 32 --s3-chunk-size 128000 --fast-list -P

!rclone sync 1d:AI/automatic1111/models/faceswaplab /content/stable-diffusion-webui/models/faceswaplab --s3-upload-concurrency 32 --s3-chunk-size 128000 --fast-list -P

!rclone sync 1d:AI/automatic1111/models/adetailer /content/stable-diffusion-webui/models/adetailer --s3-upload-concurrency 32 --s3-chunk-size 128000 --fast-list -P

!rclone sync 1d:AI/automatic1111/extensions /content/stable-diffusion-webui/extensions --s3-upload-concurrency 32 --s3-chunk-size 128000 --fast-list -P

!wget https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P /content/stable-diffusion-webui/models/VAE


In [ ]:
!7z x /content/stable-diffusion-webui/extensions.7z -o/content/stable-diffusion-webui


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,8 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/stable-diffusion-webui/                                          1 file, 147169194 bytes (141 MiB)

Extracting archive: /content/stable-diffusion-webui/extensions.7z
--
Path = /content/stable-diffusion-webui/extensions.7z
Type = 7z
Physical Size = 147169194
Headers Size = 44755
Method = LZMA2:24 BCJ
Solid = +
Blocks = 2

  0%     16% 827 - extensions/091sd-webui-faceswaplab/docs/assets/images/example1.png                                                                              18% 843 - extensio

In [ ]:
!rclone sync 1d:AI/automatic1111/extensions /content/stable-diffusion-webui/extensions --s3-upload-concurrency 32 --s3-chunk-size 128000 --fast-list -P

In [ ]:
!rm -rf /content/stable-diffusion-webui/extensions

In [ ]:
#@title ## Step 3: Download Creations Folder

#@markdown This step is manual. **You'll need to interrupt execution** on the menu bar at the top, select:

#@markdown > Runtime > Interrupt execution

#@markdown Interrupting execution will stop stable-diffusion-webui so make sure you're done using it.

filename = '/content/output.zip'
!zip -r "{filename}" /content/stable-diffusion-webui/outputs
from google.colab import files
files.download(filename)

In [ ]:
!pip install git+https://github.com/demotomohiro/remocolab.git
import remocolab
remocolab.setupSSHD()

  Cloning https://github.com/demotomohiro/remocolab.git to /tmp/pip-req-build-wnlsk6tp
  Running command git clone --filter=blob:none --quiet https://github.com/demotomohiro/remocolab.git /tmp/pip-req-build-wnlsk6tp
  Resolved https://github.com/demotomohiro/remocolab.git to commit 176445b2ca07cacf3a41036cf44bb38caf2c5049
  Preparing metadata (setup.py) ... done
---
As ngrok doesn't work on colab for long time, default tunnel method has been changed to Argo tunnel.
Please read this for more details:
https://github.com/demotomohiro/remocolab/blob/master/README.md


Label(value='')

FloatProgress(value=0.0, layout=Layout(border='1px solid #118800'), max=1.0)

openssh-server is already installed
Failed to download  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64


OSError: [Errno 26] Text file busy: 'cloudflared'

In [ ]:
!pip install xformers

  Using cached torch-2.2.0-cp310-cp310-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl.metadata (1.8 kB)
  Using cached triton-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
Using cached torch-2.2.0-cp310-cp310-manylinux1_x86_64.whl (755.5 MB)
Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
Using cached triton-2.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (167.9 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.1.0
    Uninstalling triton-2.1.0:
      Successfully uninstalled triton-2.1.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.18.1
    Uninstalling nvidia-nccl-cu12-2.18.1:
      Successfully uninstalled nvidia-nccl-cu12-2.18.1
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0
    Uninstalling torch-2.1.0:
      Successfully uninstalled t

In [ ]:
!pip install tensorrt
!pip install torchvision

  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1.post1-py2.py3-none-any.whl size=17283 sha256=f404533acb341d762314bf6032ab412b386ea219d63dd3910f2f08cb7d5043e7
  Stored in directory: /root/.cache/pip/wheels/f4/c8/0e/b79b08e45752491b9acfdbd69e8a609e8b2ed7640dda5a3e59
Successfully built tensorrt
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.19.3
    Uninstalling nvidia-nccl-cu12-2.19.3:
      Successfully uninstalled nvidia-nccl-cu12-2.19.3
  Attempting uninstall: torch
    Found existing installation: torch 2.2.0

In [ ]:
!rclone copy 1d:AI/automatic1111/extensions /content/stable-diffusion-webui/extensions --s3-upload-concurrency 32 --s3-chunk-size 128000 --fast-list -P


In [ ]:
!cp -R /content/drive/MyDrive/loras /content/stable-diffusion-webui/models/Lora

^C


In [ ]:
!rm -rf /content/stable-diffusion-webui/models/Lora 1d:AI/automatic1111/models/lora -P

rm: invalid option -- 'P'
Try 'rm --help' for more information.


In [ ]:
!rclone sync /content/stable-diffusion-webui/models/Lora 1d:AI/automatic1111/models/lora -P

In [ ]:
 /content/drive/MyDrive/loras/* /content/stable-diffusion-webui/models/Lora

In [ ]:
!rclone sync /content/stable-diffusion-webui/outputs 1d:output2 -P

In [ ]:
!cp -R /content/drive/MyDrive/loras /content/stable-diffusion-webui/models/Lora

^C


In [ ]:
!rclone sync /content/stable-diffusion-webui/outputs 1d:output2 -P

In [ ]:
!rclone sync /content/stable-diffusion-webui/outputs 1d:output2 -P